# Projeto 2 - Ciência dos Dados

Nome:Edivaldo Rangel

Nome:Henrique Feola

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import numpy as np
import math
import os.path
import pandas as pd
import json
from random import shuffle
import string
import os


___
## Autenticando no  Twitter

* Conta: @Estudan83556905

In [6]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
#with open('auth.pass') as fp:    
 #   data = json.load(fp)

#Configurando a biblioteca. Não modificar
#auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
#auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [7]:
#Produto escolhido:
produto = 'it 2'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [9]:
#Cria um objeto para a captura
#api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
#i = 1
#msgs = []
#msgs_filtradas = []
#for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
 #   msgs.append(msg.full_text.lower())
  #  i += 1
   # if i > 1200:
    #    break
        
#msgs_filtradas = set(msgs)

#Tweet = []
#for tweet in msgs_filtradas:
 #   Tweet.append(tweet)
  #  if len(Tweet) >= n:
   #     break
    
#Embaralhando as mensagens para reduzir um possível viés

#shuffle(Tweet)
#print(len(Tweet))

Salvando os dados em uma planilha Excel:

In [11]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
#if not os.path.isfile('./{0}.xlsx'.format(produto)):
 #   
    #Abre o arquivo para escrita
  #  writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
   # dft = pd.DataFrame({'Treinamento' : pd.Series(Tweet[:t])})
    #dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

   # dfc = pd.DataFrame({'Teste' : pd.Series(Tweet[t:])})
   # dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    #writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [3]:
import re 

def limpa(texto):
    pontuacao = '[!\-.:?;]'
    padrao = re.compile(pontuacao)
    texto_limpo = re.sub(padrao, ' ', texto)
    return texto_limpo
def replace(coluna):
    coluna = coluna.replace("https://","").replace("."," ").replace(":"," ").replace(","," ").replace("'"," ")\
    .replace('"', " ").replace("#","").replace("-","").replace("•", "").replace("\n\n", " ").replace("\n", " ")\
    .replace("—", " ").replace("rt", "").replace("?", " ").replace("!", " ").replace("/","").replace(";", " ")\
    .replace("{", "").replace("}", "").replace("|","").replace("(", "").replace(")", "").replace("_", "")\
    .replace("__", "").replace("\\","").replace("´", "").replace("ˆ","").replace("˜","").replace("\\", " \\").replace("U+", " U+")
    return coluna

In [4]:
dados_treino = pd.read_excel('it 2.xlsx', sheet_name = 'Treinamento')
dados_teste = pd.read_excel('it 2.xlsx', sheet_name = 'Teste')
dados_idx = dados_treino.set_index('Treinamento')
dados_idx_teste = dados_teste.set_index('Classificacao')
dados_relevante = dados_idx[dados_idx.Classe == 1]
dados_irrelevante = dados_idx[dados_idx.Classe == 0]

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
tweets = str((dados_idx.index))
tweets_teste = dados_idx_teste.values.tolist()
tweets_re = str((dados_relevante.index))
tweets_ir = str((dados_irrelevante.index))

Limpando os tweets:

In [6]:
tweets_limpo = limpa(tweets.lower())
tweets_re_limpo = limpa(tweets_re.lower())
tweets_ir_limpo = limpa(tweets_ir.lower())

In [7]:
universo_raw = pd.Series(tweets_limpo.split())
relevantes_raw =  pd.Series(tweets_re_limpo.split())
irrelevantes_raw =  pd.Series(tweets_ir_limpo.split())
## retirando os \n
universo = replace(universo_raw)
relevantes = replace(relevantes_raw)
irrelevantes = replace(irrelevantes_raw) 

In [8]:
tabela_universo = universo.value_counts(True)
tabela_relevante = relevantes.value_counts(True)
tabela_irrelevante = irrelevantes.value_counts(True)
tabela_relevante_abs = relevantes.value_counts()
tabela_irrelevante_abs = irrelevantes.value_counts()

In [9]:
Probabilidade_relevantes = len(tabela_relevante)/len(tabela_universo)
Probabilidade_irrelevantes = len(tabela_irrelevante)/len(tabela_universo)

In [10]:
def classifica(tweet,a,b):
    if a>b:
        return 1
    else:
        return 0 
Lista_relevante_final = []
Lista_irrelevante_final = []
Classificacao = []
for tweet in tweets_teste:
    for frase in tweet:
        lista_rele = []
        lista_irrele = []
        frase_limpa_raw = limpa(frase.lower())
        frase_limpa = frase_limpa_raw.strip()
        palavras = frase_limpa.split()
         # Laplace
        prob_relevante = (tabela_relevante_abs[palavras].fillna(0) + 1)/(len(set(universo)) + len(relevantes))
        prob_irrelevante = (tabela_irrelevante_abs[palavras].fillna(0) +1)/(len(set(universo)) + len(irrelevantes))
        lista_rele.append(prob_relevante)
        lista_irrele.append(prob_irrelevante)
        Lista_relevante_final.append(lista_rele)
        Lista_irrelevante_final.append(lista_irrele)
# Naive Bayes

        for frase in Lista_relevante_final:
            for palavra in frase:
                Prob_rel = palavra.prod() *Probabilidade_relevantes   
        for frase in Lista_irrelevante_final:
            for palavra in frase:
                Prob_irrel = palavra.prod() * Probabilidade_irrelevantes
        Classific = classifica(tweet, Prob_rel,Prob_irrel) 
        Classificacao.append(Classific)

C:\Users\User\Documents\Python Scripts\python dowload\lib\site-packages\pandas\core\series.py:951: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
dados_teste["Simulacao"] = Classificacao
dados_teste.to_excel("it_2_classificado.xlsx")
dados_teste

ValueError: ZIP does not support timestamps before 1980

In [13]:
pos = 0
falso_pos = 0
neg = 0
falso_neg = 0
total = len(dados_teste["Classificacao"])

for [e,i] in zip(dados_teste["Classificacao"], dados_teste["Simulacao"]):
    if e == 1:
        if i == 1:
            pos += 1
        else: 
            falso_neg += 1
    if e == 0:            
        if i == 0:
            neg +=1
        else:
            falso_pos +=1

In [14]:
print("Verdadeiros relevantes: ", round((pos/total)*100,2), "%")
print("Falsos relevantes: ", round((falso_pos/total)*100, 2), "%")
print("Verdadeiros irrelevantes: ", round((neg/total)*100,2), "%")
print(f"Falsos irrelevantes: ", round((falso_neg/total)*100, 2), "%")
print("_"*35)

print("Acertos: ", round(((pos/total)+(neg/total))*100,2), "%")
print("Erros: ", round(((falso_pos/total)+(falso_neg/total))*100,2), "%")

Verdadeiros relevantes:  22.0 %
Falsos relevantes:  38.5 %
Verdadeiros irrelevantes:  23.0 %
Falsos irrelevantes:  16.5 %
___________________________________
Acertos:  45.0 %
Erros:  55.0 %


___
### Concluindo

### Em conclusão do nosso projeto 2, foi posivel analisar que, mesmo que menos de metade dos dados tenham sido confirmados pelo data base, os comentarios analisados sobre este tipo de genero cinematico possui muito mais comentários irrelevantes para uma concentração e analise de dados, apontando que se alguem quiser ter uma boa base para retratar o feedback popular sobre o filme ou qualquer outro tipo de entretenimento, este tipo de rede social não é a melhor plataforma. Em suma, a rede social do twitter pode não ser o melhor lugar para fazer uma pesquisa detalhada.

### Propondo diferentes cenarios para Naive Bayes

#### Como o teorema de Bayes trata de probabilidades condicionais, em que, ambos os termos analisados são condicionalmente independentes, podemos supor que a analise de dados comutativos assim como na analise do projeto podem ser evidenciadas neste aspecto.
#### Um exemplo de aplicação de Bayes fora dos procedimentos do projeto pode ser o exemplo de diagnósticos médicos, em que, dado a propabilidade do individuo ter certa doença ou prè-disposição a tal doença, a totalidade da pessoa efetivamente estar doente pode ser calculada com esta teorema

### Não é recomendado utilizar o classificador para gerar novas amostras de treinamento

#### Apesar de ser muito bom para fazer previsões em tempo real e necessitar de poucos dados para realizar uma classificação, ele não tem facilidade em relacionar fatores e seguir com padrões de comportamento, ou seja, ele pode não identificar certos padrões que seriam necessarios para a classificação efetiva dos termos analisados, principalmente na área de tweets gerados randomicamente e com emojis e caracteres especiais

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**